Credit: Đoàn Vĩnh Nhân - Toán Tin K68 - 20237376

### Phương pháp khử Gauss
+ Input: ma trận $A_{m\times n}, B_{m\times q}$
+ Output: ma trận $X_{n\times (q+k)}$, với $k$ là số cột bằng 0 sau khi khử $AB$. Các phần tử trên cột q đặc trưng cho sự phụ thuộc của các phần tử còn lại của $X$ vào các phần tử tự do.

Ví dụ với nghiệm:
$$\begin{array}{ccccc}
&&&&X_3&X_4\\
&1&2&3&4&5\\
&2&3&4&5&6\\
&3&4&5&6&7
\end{array}$$
Khi đó $X=\begin{bmatrix}X_1&X_2&X_3&X_4&X5\end{bmatrix}^\top$
\begin{array}{ll}
X_1&=\begin{pmatrix}1&2&3\end{pmatrix} + 4X_4 + 5X_5\\
X_2&=\begin{pmatrix}2&3&4\end{pmatrix} + 5X_4 + 6X_5\\
X_3&=\begin{pmatrix}3&4&5\end{pmatrix} + 6X_4 + 7X_5\\
X_4&=\begin{pmatrix}t_{41}&t_{42}&t_{43}\end{pmatrix}, \quad t \in \mathbb{R}\\
X_5&=\begin{pmatrix}t_{51}&t_{52}&t_{53}\end{pmatrix}, \quad t \in \mathbb{R}\\
\end{array}
Thuật toán
1. Khử ma trận $AB$
   + Bước 1: Tạo ma trận mở rộng $A|B$, tạo 1 danh sách index: lưu vị trí cột của các phần tử khác 0 đầu tiên trong từng hàng. Hoặc nếu là hàng đó bằng 0, lưu $n$
   + Bước 2: Sắp xếp lại index theo thứ tự không giảm, sắp xếp các hàng của ma trận mở rộng AB theo index => Để có ma trận bậc thang
   + Step 3: Khử ma trận, theo từng hàng => nếu chưa đến hàng cuối quay lại Step 2, nếu xong chuyển đến B2
2. Kiểm tra index của AB, nếu tồn tại phần tử chỉ đến cột của B trong AB => Kết luận vô nghiệm và dừng (trường hợp $rank(A)<rank(AB)$, hàng A bằng 0 nhưng hàng B có phần tử khác 0)
3. Chuyển các cột có phần tử trên đường chéo chính bằng $0$ của $A$ ra sau $B$
   + Bước 1: Dùng danh sách index, tìm kiếm những cột có phần tử trên đường chéo chính bằng 0
   + Bước 2: Chuyển các cột đó ra sau ma trận B
4. Giải tìm $X$ với $A$ mới và $B$ mới (Ma trận A giờ đây là ma trận tam giác trên)
   + Bước 1: Tạo ma trận không $X_{n\times (q+k)}$
   + Bước 2: Dùng vòng lặp col và row, tính toán từng phần tử của ma trận $X$

Ví dụ:
     $\begin{bmatrix}4&0&0\\0&2&1\\0&0&1\\5&2&1\\0&0&0\\0&3&2\end{bmatrix}$, index $= [0 ,1 ,2, 0, 3,1] $
$\Rightarrow$ Sắp xếp lại:
     $\begin{bmatrix}4&0&0\\5&2&1\\0&2&1\\0&3&2\\0&0&1\\0&0&0\end{bmatrix}$
          index $=[0 ,0 ,1, 1, 2, 3]$

In [1]:
from sympy import Matrix, zeros, N
import numpy as np

# Hàm này được thêm vào để loại bỏ các giá trị nhỏ hơn 10⁻¹⁰, đảm bảo chúng được coi là 0.
def chop_small_values(mat):
    if isinstance(mat, Matrix):
        return mat.applyfunc(lambda x: 0 if abs(x) < 1e-10 else x)
    else:
        return 0 if abs(mat) < 1e-10 else mat
#Danh sách các vị trí khác 0 đầu tiên của từng hàng, dùng để sắp xếp AB theo cấu trúc bậc thang
def indexpivot(AB):
    rowAB, colAB = AB.shape
    index = []
    for row in range(rowAB):
        for col in range(colAB):
            if abs(AB[row, col]) > 1e-10: 
                index.append(col)
                break
            if col == colAB - 1:
                index.append(col + 1)
    return index
    
#Vị trí các cột có phần tử ở đường chéo chính bằng 0
def missingindex(A):
    colA = A.shape[1]
    index = indexpivot(A)
    return list(set(range(colA)) - set(index))
    
#Khử gauss
def gauss(A, B):
    A, B = Matrix(N(A)), Matrix(N(B))
    AB = chop_small_values(A.row_join(B))
    rowAB, colAB = AB.shape
    
    #Sắp xếp lại AB theo cấu trúc bậc thang trong trường hợp input không phải là bậc thang    
    index = indexpivot(AB)
    sorted_order = sorted(range(len(index)), key=lambda k: index[k])
    index = sorted(index)
    AB = chop_small_values(AB[list(sorted_order), :])
    display(AB)

    #Khử Gauss (Vừa khử, vừa sắp xếp lại AB theo cấu trúc bậc thang)
    for i in range(len(index)):
        if index[i] != colAB:
            for j in range(i + 1, len(index)):
                AB[j, :] = chop_small_values(Matrix(AB[j, :] - AB[i, :] / AB[i, index[i]] * AB[j, index[i]]))
            index = indexpivot(AB)
            sorted_order = sorted(range(len(index)), key=lambda k: index[k])
            index = sorted(index)
            AB = chop_small_values(AB[list(sorted_order), :])
            print("Bước khử")
            display(AB)
    print("Ma trận sau khi khử:")
    display(AB)
    return AB

#Giải AX=B
def gauss_solve(A, B):
    #Ma trận đã khử AB
    A, B = Matrix(N(A)), Matrix(N(B))
    AB = gauss(A, B)

    #Kiểm tra trường hợp vô nghiệm
    for i in indexpivot(AB):
        if i >= A.shape[1] and i < AB.shape[1]:
            print("Hệ vô nghiệm")
            return 

    #Ma trận A, B mới từ ma trận đã khử    
    A = AB[:, :A.shape[1]]
    B = AB[:, A.shape[1]:]
    colB = B.shape[1]

    #Chuyển những cột có phần tử trên đường chéo chính bằng 0 sang B
    missindex = missingindex(A)
    if missindex:
        print(f"index: {[x + 1 for x in indexpivot(A)]}")
        print(f"missing index: {[x + 1 for x in missindex]}")
        for i in missindex:
            move_col = -1 * A[:, i]
            B = B.row_join(move_col)
        A = A[:, [i for i in range(A.shape[1]) if i not in missindex]]
        print("Ma trận sau khi chuyển cột:")
        display(chop_small_values(A.row_join(B)))

    #Tính toán ma trận X    
    X = zeros(A.shape[1], B.shape[1])
    rowX, colX = X.shape
    index = indexpivot(A)
    
    for col in range(colX):
        for row in range(rowX - 1, -1, -1):
            sum_val = sum(A[row, k] * X[k, col] for k in range(index[row], rowX))
            X[row, col] = chop_small_values((B[row, col] - sum_val) / A[row, index[row]])
    
    print(f"Số phần tử tự do là {len(missindex)}: X{[x + 1 for x in missindex]}. Nghiệm:")
    return X

In [2]:
A = Matrix(np.loadtxt("G_input_A.txt"))
print("A =")
display(A)
print("B =")
B = Matrix(np.loadtxt("G_input_B.txt"))
display(B)
print("A|B=")
X = gauss_solve(A,B)
display(X)

A =


Matrix([
[ 130.75,    3.75,    5.25,    11.25],
[    0.0,     0.0,     0.0,      0.0],
[    0.0,     0.0,     0.0,      0.0],
[16.5714, 14.8571, 25.4286, 140.4286]])

B =


Matrix([
[ 310.2275],
[      0.0],
[      0.0],
[-165.0229]])

A|B=


...

Matrix([
[130.75,             3.75,             5.25,            11.25,          310.2275],
[     0, 14.3818208413002, 24.7632091778203, 139.002762523901, -204.341477388145],
[     0,                0,                0,                0,                 0],
[     0,                0,                0,                0,                 0]])

index: [1, 2, 5, 5]
missing index: [3, 4]
Ma trận sau khi chuyển cột:


Matrix([
[130.75,             3.75,          310.2275,             -5.25,            -11.25],
[     0, 14.3818208413002, -204.341477388145, -24.7632091778203, -139.002762523901],
[     0,                0,                 0,                 0,                 0],
[     0,                0,                 0,                 0,                 0]])

Số phần tử tự do là 2: X[3, 4]. Nghiệm:


Matrix([
[ 2.78018119153851, 0.00923062220984498, 0.191161701735659],
[-14.2083175449759,    -1.7218410277166, -9.66517133384996]])